In [1]:
# Import the datetime module from the datetime library.
from datetime import datetime
import pandas as pd
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [3]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [4]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

    # Add 1 to the record count.
    record_count += 1

Processing Record 1 of Set 1 | narsaq
Processing Record 2 of Set 1 | klaksvik
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | busselton
Processing Record 5 of Set 1 | saint george
Processing Record 6 of Set 1 | kargil
Processing Record 7 of Set 1 | avarua
Processing Record 8 of Set 1 | lagoa
Processing Record 9 of Set 1 | mataura
Processing Record 10 of Set 1 | margate
Processing Record 11 of Set 1 | bluff
Processing Record 12 of Set 1 | grindavik
Processing Record 13 of Set 1 | bredasdorp
Processing Record 14 of Set 1 | vangaindrano
Processing Record 15 of Set 1 | vestmannaeyjar
Processing Record 16 of Set 1 | kavieng
Processing Record 17 of Set 1 | kazalinsk
City not found. Skipping...
Processing Record 18 of Set 1 | babanusah
City not found. Skipping...
Processing Record 19 of Set 1 | ushuaia
Processing Record 20 of Set 1 | peno
Processing Record 21 of Set 1 | atherton
Processing Record 22 of Set 1 | lata
Processing Record 23 of Set 1 | banjar
Processing Record 

In [8]:


# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Narsaq,60.9167,-46.0500,38.28,92,100,6.71,GL,2021-09-18 15:07:03
1,Klaksvik,62.2266,-6.5890,54.52,85,100,17.20,FO,2021-09-18 15:07:04
2,Albany,42.6001,-73.9662,77.74,84,37,5.44,US,2021-09-18 15:06:01
3,Busselton,-33.6500,115.3333,55.09,85,99,4.99,AU,2021-09-18 15:07:04
4,Saint George,37.1041,-113.5841,79.27,40,1,1.99,US,2021-09-18 15:07:05


In [10]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")